# Baseline-net

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.0-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleus

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

## Loading the data

In [1]:
from time import time
import os
import gzip
import numpy as np
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from keras.datasets import cifar10

Using TensorFlow backend.


In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, 
                                                  random_state=1974)

In [0]:
train_features = X_train.reshape(X_train.shape[0], 32, 32, 3)/255
vali_features = X_val.reshape(X_val.shape[0], 32, 32, 3)/255
test_features = X_test.reshape(X_test.shape[0], 32, 32, 3)/255

y_train_cat = np_utils.to_categorical(y_train)
y_val_cat = np_utils.to_categorical(y_val)
y_test_cat = np_utils.to_categorical(y_test)

## Training the model

In [0]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input
from keras import optimizers
from keras.callbacks import TensorBoard, ReduceLROnPlateau, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras import backend as K

In [0]:
class TimingCallback(keras.callbacks.Callback):
  """Callback that saves the time elapsed of each epoch to the log.
  """  
  def on_epoch_begin(self, epoch, logs={}):
    self.starttime=time()
  def on_epoch_end(self, epoch, logs={}):
    logs['time_elapsed'] = (time()-self.starttime)

In [0]:
img_rows, img_cols = 32, 32
input_shape = (img_rows, img_cols, 3)

num_classes_fine = 10

img_input = Input(shape=input_shape, name='input')


x = Conv2D(128, (3, 3), activation='relu', name='block1_conv1', padding='same')(img_input)

x = Conv2D(128, (3, 3), activation='relu', name='block1_conv2', padding='same')(x)
x = MaxPooling2D((2, 2), name='block2_pool')(x)

x = Conv2D(128, (3, 3), activation='relu', name='block1_conv3', padding='same')(x)
x = MaxPooling2D((2, 2), name='block3_pool')(x)

x = Flatten(name='flatten')(x)
x = Dense(512, name='fc_1')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

x = Dense(512, name='fc_2')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

fine_pred = Dense(num_classes_fine, activation='softmax', name='predictions_fine')(x)

In [20]:
model = Model(inputs=img_input, outputs= fine_pred, name='baseline_net')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 128)       3584      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
_________________________________________________________________
block1_conv3 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block3_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
__________

In [0]:
adam = optimizers.Adam(lr=0.00025)
model.compile(loss='categorical_crossentropy', 
              optimizer=adam,
              metrics=['accuracy'])

In [0]:
clr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
time_cb = TimingCallback()
csv_cb = CSVLogger('drive/TCC-ITAU/cifar-10/training-data/baseline/baseline-net-simple.csv', separator=',', append=False)


cbks = [clr_cb, time_cb, csv_cb]

In [23]:
epochs = 50
start_time = time()
training = model.fit(train_features, y_train_cat,
                     validation_data=(vali_features,  y_val_cat),
                     epochs=epochs,
                     batch_size=256, 
                     callbacks=cbks,
                     verbose=1)
training_time = time() - start_time

Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - 24s 607us/step - loss: 1.8815 - acc: 0.3665 - val_loss: 1.5222 - val_acc: 0.4579
Epoch 2/50
40000/40000 [==============================] - 22s 556us/step - loss: 1.3448 - acc: 0.5272 - val_loss: 1.3262 - val_acc: 0.5408
Epoch 3/50
40000/40000 [==============================] - 22s 555us/step - loss: 1.0864 - acc: 0.6168 - val_loss: 0.9993 - val_acc: 0.6349
Epoch 4/50
40000/40000 [==============================] - 22s 556us/step - loss: 0.9301 - acc: 0.6718 - val_loss: 0.9167 - val_acc: 0.6692
Epoch 5/50
40000/40000 [==============================] - 22s 556us/step - loss: 0.8124 - acc: 0.7133 - val_loss: 0.8564 - val_acc: 0.7001
Epoch 6/50
40000/40000 [==============================] - 22s 556us/step - loss: 0.7229 - acc: 0.7439 - val_loss: 0.8823 - val_acc: 0.6945
Epoch 7/50
40000/40000 [==============================] - 22s 556us/step - loss: 0.6301 - acc: 0.7779 - val_loss: 0.

In [24]:
model.evaluate(test_features, y_test_cat)

10000/10000 [==============================] - 3s 273us/step


[1.077336925649643, 0.7733]

In [25]:
print('Total training time: {}'.format(training_time))

Total training time: 1137.8237504959106


In [0]:
model.save('drive/TCC-ITAU/cifar-10/models/baseline.h5')